In [7]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA


c:\Python312\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
c:\Python312\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


In [8]:
# # Fonction pour télécharger des données historiques
# def download_data(ticker, start_date, end_date):
#     data = yf.download(ticker, start=start_date, end=end_date)
#     data['Date'] = data.index
#     return data

# # Exemple de téléchargement de données
# ticker = 'AAPL'  # Symbole boursier de l'action Apple
# start_date = '2020-01-01'
# end_date = '2023-01-01'
#/////////////////////////////////////////////
# def read_data_from_csv(filename):
#     data = pd.read_csv(filename)
#     data['Date'] = pd.to_datetime(data['Date'])
#     data.set_index('Date', inplace=True)
#     return data

# # Exemple de lecture de données à partir d'un fichier CSV local
# filename = 'aapl_5min.csv'
# data = read_data_from_csv(filename)

# # data = download_data(ticker, start_date, end_date)
# print(data.head())
# def download_data(ticker, start_date, end_date):
#     data = yf.download(ticker, start=start_date, end=end_date)
#     data['Date'] = data.index
#     return data

# # Exemple de téléchargement de données
# ticker = 'AAPL'  # Symbole boursier de l'action Apple
# start_date = '2020-01-01'
# end_date = '2023-01-01'

# data = download_data(ticker, start_date, end_date)
# print(data.head())

# # Sauvegarder les données dans un fichier CSV
# data.to_csv('historical_data.csv', index=False)

# Fonction pour lire des données historiques depuis un fichier CSV
def read_data_from_csv(filename):
    data = pd.read_csv(filename)
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    data.set_index('Datetime', inplace=True)
    return data

# Exemple de lecture de données à partir d'un fichier CSV local
filename = './data/aapl_5min.csv'  # Remplacez par le nom de votre fichier CSV
data = read_data_from_csv(filename)
print(data.head())


                                 Open        High                 Low  \
Datetime                                                                
2024-05-01 09:30:00-04:00  170.000000  170.270004  169.11000061035156   
2024-05-01 09:35:00-04:00  169.110001  169.839996  169.11000061035156   
2024-05-01 09:40:00-04:00  169.669998  170.360001  169.56500244140625   
2024-05-01 09:45:00-04:00  170.279999  170.560303  170.17999267578125   
2024-05-01 09:50:00-04:00  170.539993  170.638702  170.27499389648438   

                                Close   Adj Close   Volume  
Datetime                                                    
2024-05-01 09:30:00-04:00  169.110901  169.110901  2334916  
2024-05-01 09:35:00-04:00  169.675003  169.675003   618720  
2024-05-01 09:40:00-04:00  170.250000  170.250000   656942  
2024-05-01 09:45:00-04:00  170.549896  170.549896   605474  
2024-05-01 09:50:00-04:00  170.544998  170.544998   497845  


In [9]:
#definitions des indicateurs
def RSI(data, period):
    series = pd.Series(data)
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))


def MACD(data, short_period=12, long_period=26, signal_period=9):
    short_ema = pd.Series(data).ewm(span=short_period, adjust=False).mean()
    long_ema = pd.Series(data).ewm(span=long_period, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def EMA(data, period):
    return data.ewm(span=period, adjust=False).mean()

def SMA(data, period):
    return data.rolling(window=period).mean()


In [10]:
params1 = 20
params2 = 50
params3 = 30

In [11]:
class Strat(Strategy):
    parametre1 = params1  # exemple de valeur par défaut
    parametre2 = params2  # exemple de valeur par défaut

    def init(self):
        # Initialisation des moyennes mobiles exponentielles
        self.ema1 = self.I(EMA, pd.Series(self.data.Close), self.parametre1)
        self.ema2 = self.I(EMA, pd.Series(self.data.Close), self.parametre2)

    def next(self):
        # Croisement des EMA pour les signaux d'achat et de vente
        if crossover(self.ema1, self.ema2):
            self.buy()
        elif crossover(self.ema2, self.ema1):
            self.sell()



In [15]:
def perform_backtest(data):
    Strat.parametre1 = params1
    Strat.parametre2 = params2
    print(Strat.parametre1, Strat.parametre2)
    bt = Backtest(data, Strat, cash=10000, commission=.002)
    output = bt.run()
    bt.plot()
    return output

# Exécution du backtest
result = perform_backtest(data)
print(result)


20 50


c:\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Python312\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Python312\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(resample_rule, label='left')


TypeError: '<=' not supported between instances of 'float' and 'str'